<a href="https://colab.research.google.com/github/seulgiKim95/WordCloud/blob/Daehyeon/Wanted_4th_%EA%B8%88%EC%9A%94%EC%9D%BC%EC%97%90_%EB%A7%8C%EB%82%98%EC%9A%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
# 이 페이지를 열었을 때 마다 실행해야 됩니다. 
# 코랩은 로컬환경(내 pc에 저장소)에 설치한 라이브러리를 저장하지 않기 때문입니다.
# 실행 후 아래 코드는 주석 처리 해주세요. 주석은 필요부분을 블록처리(드래그나 [Shift]키로 지정)
# 한 영역에 [Ctrl] + [/] 키를 누르면 주석 처리가 됩니다. (한 줄 단위도 됩니다.)
!pip install beautifulsoup4 #뷰티플수프 라이브러리
!pip install selenium #셀리니움 라이브러리
!apt-get update #크롬브라우저는 계속 업데이트가 되기 때문에 자동으로 업데이트를 할 수 있게 해줍니다.
!apt install chromium-chromedriver #셀레니움 이용시 크롬 드라이버가 꼭 필요합니다.
!cp /usr/lib/chromium-browser/chromedriver /usr/bin 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://develope

In [127]:
## 필요한 라이브러리 import

# -*- coding: UTF-8 -*-
import time #시간지연을 해주기 위함입니다.
import pandas as pd #pandas가 추가 되었습니다.
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import json


In [128]:
options = webdriver.ChromeOptions() #기본 설정이라고 생각하시면 됩니다. 바로 실행 ㄱㄱ
options.add_argument('--headless')        ## Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

In [129]:
base_url = "https://www.wanted.co.kr/" #저희는 '원티드' 사이트를 조사하기로 했죠? 
                                       # 만약 다른 사이트를 할 경우 여기 다른 사이트주소를 입력합니다.
keyword = "데이터 분석가" # 저희가 찾을 직무를 입력합니다
                          # 데이터 엔지니어, 데이터 분석가, 데이터 사이언티스트, 머신러닝 엔지니어, 데이터 플랫폼 개발자
                          # DBA, 임베디드, IoT, 마켓팅 팀의 데이터 전문가, 정보보안, 백엔드, 풀스택 개발자
                          # 띄어쓰기 포함입니다.

In [130]:
## 해당 url로 이동
driver = webdriver.Chrome('chromedriver', options=options)
driver.get(base_url) #base_url변수에는 위에 '원티드'사이트 URL이 들어있습니다.

## 사이트에서 검색어 입력 부분(검색어 창)을 찾아 xpath로 넣어 찾아준다. 사이트에서 돋보기 부분 해당
driver.find_element('xpath','//*[@id="__next"]/div[1]/div/nav/aside/ul/li[1]/button').send_keys(Keys.ENTER)

#search_job = driver.find_element('xpath','//*[@id="__next"]/div[1]/div[2]/div/form/input') # 돋보기를 누른 뒤 검색창에 해당
search_job = driver.find_element('xpath','//*[@id="__next"]/div[1]/div[2]/div/form/input')

## 찾은 검색어 창에 검색어를 입력하고 enter를 친다.
search_job.send_keys(keyword+Keys.ENTER) #keyword는 '직무'가 담겨져 있습니다. 
                                         #현재 keyword에는 '데이터 분석가'가 들어가 있고
                                         #Keys.ENTER로 검색합니다.
                                         #검색창에 데이터 전문가 라고 치고 엔터키 누른 것과 동일

## 페이지가 로딩 되는 동안 기다려준다.(여기서는 5초) #안해주면 사이트를 읽어 올 수가 없습니다.
time.sleep(5) #단위는 초                                       #꼭 5초가 아니어도 되지만 여유 있게 시간을 줍시다.

# 페이지 스크롤 다 내리기
prev_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 스크롤을 화면 가장 아래로 내린다
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    
    # 페이지 로딩 대기
    time.sleep(2)

    # 현재 문서 높이를 가져와서 저장
    curr_height = driver.execute_script("return document.body.scrollHeight")

    if(curr_height == prev_height):
        break
    else:
        prev_height = driver.execute_script("return document.body.scrollHeight")

## 페이지 소스를 html 변수에 저장하고 driver를 닫아준다.
html = driver.page_source #여기에는 우리가 엔터를 쳐서 들어간 페이지의 모든 html정보가 들어갑니다.
driver.quit() #파일 입출력 때 f.close()와 비슷하다고 생각해주세요.

In [46]:

# ## 페이지 소스를 html 변수에 저장하고 driver를 닫아준다.
# html = driver.page_source #여기에는 우리가 엔터를 쳐서 들어간 페이지의 모든 html정보가 들어갑니다.
# driver.quit() #파일 입출력 때 f.close()와 비슷하다고 생각해주세요.

In [47]:
# 궁금하시면 주석을 지우고 확인해 보세요.
#print(type(html))
#print(html)

<class 'str'>
<html lang="ko-KR" class="ko kr"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=no"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta http-equiv="cache-control" content="no-store"><meta property="og:type" content="website"><meta property="og:locale" content="ko_KR"><meta property="fb:app_id" content="316787678519888"><meta name="theme-color" content="#ffffff"><meta name="google-play-app" content="app-id=com.wanted.android.wanted"><meta name="apple-itunes-app" content="app-id=1074569961"><meta name="msapplication-TileColor" content="#ffffff"><meta name="msapplication-TileImage" content="//static.wanted.co.kr/favicon/144x144.png"><meta name="naver-site-verification" content="ef4f7f8504106401fe915b922514a5b352f59ca2"><meta name="google-site-verification" content="KEXLzgQBOVNqw8uNBWOXYOz2L4vUApbKxqxV6twdkxQ"><title>데이터 분석가 | 원티드 통합검색</title><meta property="og:image" content="https://static.wanted.co.kr/

In [131]:
## 페이지 소스를 beautifulsoup 객체로 만들어준다.
page_soup = BeautifulSoup(html, 'html.parser') #html.parser는 'html을 분석한다'고 생각하시면 됩니다.
                                               #여기서부터는 BeautifulSoup가 쓰이기 시작합니다.

In [132]:
## 우리가 필요한 데이터가 있는 곳의 태그와 class 이름을 찾아서 모두 찾아준다.
#'데이터 분석가를 검색했을 때 각 회사 정보가 뜹니다. 자세한 정보를 보려면 각 회사의 이미지(링크)를
#클릭 했어야 했습니다. 따라서 우리가 찾을 html태그는 링크 태그인 'a' 입니다.
#클래스 이름을 찾으려고 보니 class는 없고 'class aria-label=position link' 이라고 있네요.
#find_all()함수의 요소에 우리가 찾는 태그를 첫번째 인자값으로 넣고,
#두번째 인자값에는 딕셔너리 구조로 넣어주세요 '=' 이 딕셔너리 ':'에 해당됩니다. class는 지우고 'aria-label만 입력  
keyword_result = page_soup.find_all('a',{'aria-label':'position link'}) #fin_all() 만족하는 것 다 찾음 find는 하나만

#keyword_result #궁금하시면 주석 지우고 실행해보세요.

In [133]:
keyword_list = []
for i in keyword_result:
  if keyword in i.text: #.text는 태그안에 있는 텍스트만 출력해줍니다.
    ## 찾은 태그가 포함된 소스에서 href 내용만 가져와서 list에 추가해준다.
    keyword_list.append(i['href'])

In [134]:
print(keyword_list) #하나 하나가 우리가 찾아야 하는 회사입니다.

['/wd/122663', '/wd/121695', '/wd/97358', '/wd/100357', '/wd/108345', '/wd/107375', '/wd/36202', '/wd/121903', '/wd/85810', '/wd/73846', '/wd/99707', '/wd/119414', '/wd/33147', '/wd/96165', '/wd/97932', '/wd/97599', '/wd/112508', '/wd/113652', '/wd/113657', '/wd/106205', '/wd/65289', '/wd/90183', '/wd/63921', '/wd/102064', '/wd/65223', '/wd/119366', '/wd/58377', '/wd/97483', '/wd/100217', '/wd/70909', '/wd/120691', '/wd/106448', '/wd/106446', '/wd/108094', '/wd/104809', '/wd/112186', '/wd/115695', '/wd/27569', '/wd/30355', '/wd/73221', '/wd/71388', '/wd/105547', '/wd/120328', '/wd/86326', '/wd/53378', '/wd/113056', '/wd/83163', '/wd/15724', '/wd/74342', '/wd/87702', '/wd/99162', '/wd/79566', '/wd/52879', '/wd/102292', '/wd/115066', '/wd/120325', '/wd/106998', '/wd/115736', '/wd/107004', '/wd/107009', '/wd/107168', '/wd/107002', '/wd/106989', '/wd/115963', '/wd/105687', '/wd/119997', '/wd/106451', '/wd/90334', '/wd/49090', '/wd/49163', '/wd/71455', '/wd/110533', '/wd/77539']


In [135]:
len(keyword_list) #길이를 구하면 몇 건인지 알 수 있습니다.
#print(keyword_list[0]) #이것도 궁금하시면 주석 지우고 해보세요

73

In [74]:
driver = webdriver.Chrome('chromedriver', options=options)
driver.get('https://www.wanted.co.kr/wd/97358')
#driver.get('https://www.wanted.co.kr'+keyword_list[0]) 
h = driver.page_source
driver.quit()

In [75]:
com_html = BeautifulSoup(h,'html.parser')

In [76]:
print(com_html)

<html class="ko kr" lang="ko-KR"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1.0, user-scalable=no" name="viewport"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="no-store" http-equiv="cache-control"/><meta content="website" property="og:type"/><meta content="ko_KR" property="og:locale"/><meta content="316787678519888" property="fb:app_id"/><meta content="#ffffff" name="theme-color"/><meta content="app-id=com.wanted.android.wanted" name="google-play-app"/><meta content="app-id=1074569961" name="apple-itunes-app"/><meta content="#ffffff" name="msapplication-TileColor"/><meta content="//static.wanted.co.kr/favicon/144x144.png" name="msapplication-TileImage"/><meta content="ef4f7f8504106401fe915b922514a5b352f59ca2" name="naver-site-verification"/><meta content="KEXLzgQBOVNqw8uNBWOXYOz2L4vUApbKxqxV6twdkxQ" name="google-site-verification"/><title>[휴머스온] 데이터 분석가 채용 | 원티드</title><meta content="[휴머스온] 데이터 분석가 채용 | 원티드" property="og:title

In [136]:
info_list = ['주요업무', '자격요건', '우대사항', '혜택 및 복지', '기술스택 ・ 툴'] # 추후 추가 및 예외처리하기
print(info_list)

['주요업무', '자격요건', '우대사항', '혜택 및 복지', '기술스택 ・ 툴']


In [ ]:
# from google.colab import drive # 구글 드라이브에 csv파일을 저장하기 위함

# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [71]:
# f = open('/content/gdrive/My Drive/Colab Notebooks/txt_to_csv.txt', "w")
# f.close()

In [72]:
# info_list[4]

'기술스택 ・ 툴'

In [124]:
#info_tahg는 info_list에 들어있는 텍스트와 같은 h6태그를 찾음
info_tag = com_html.find('h6', string = info_list[0])
info_tag.nextSibling.text



'데이터 가공, 분석 및 시각화- 온라인로그/캠페인 데이터 등을 활용하여 비즈니스 이슈에 대해 가설을 수립- 데이터 핸들링 및 분석- 리포트 작성, 대시보드 기획/관리 등 시각화'

In [ ]:
#'h5'태그에는 회사 이름이 들어가 있습니다.
company_name = com_html.find('h5')
company_name.text

'머큐리프로젝트'

In [190]:
# empty 데이터 프레임 생성
columns=['회사이름','주요업무', '자격요건', '우대사항', '혜택 및 복지', '기술스택 ・ 툴', '지역']
df = pd.DataFrame(columns=columns)
df

,회사이름,주요업무,자격요건,우대사항,혜택 및 복지,기술스택 ・ 툴,지역


In [191]:
# for문을 사용해 회사 별 링크로 이동 한 뒤
# 다시 for문을 만나 info_list의 정보를 찾은 뒤
# 회사 이름과 위치를 찾은 다음 for문을 나간다.

for i in range(len(keyword_list)):
  info_tag_line = []
  driver = webdriver.Chrome('chromedriver', options=options)
  #driver.get('https://www.wanted.co.kr/wd/121695')
  driver.get('https://www.wanted.co.kr'+keyword_list[i]) #keyword_list증가시키며 링크 이동
  h = driver.page_source
  driver.quit()

  com_html = BeautifulSoup(h,'html.parser')

  company_name = com_html.find('h5') #회사이름

  company_loc = com_html.find('script',{'type':'application/ld+json'}) #회사 주소 불러오기
  company_loc = dict(json.loads(company_loc.text))['jobLocation']['address']['streetAddress']
 

  for j in info_list:
        info_tag = com_html.find('h6', string = j) #'주요업무', '자격요건', '우대사항', '혜택 및 복지', '기술스택 ・ 툴'
        if info_tag != None:
          if j == '기술스택 ・ 툴':
            # print(info_tag)
            # print()
            # print(keyword_list[i])
            info_tag_line.append(list(map(lambda x:x.text,info_tag.next_element.next_element.findChild().findChildren())))  
          else:  
            info_tag_line.append(info_tag.nextSibling.text)   
        else:
          info_tag_line.append(0)
     
        # print(info_tag_line)
  df = df.append(pd.Series([company_name, info_tag_line[0], info_tag_line[1], info_tag_line[2], info_tag_line[3], info_tag_line[4], company_loc],index=columns),ignore_index=True)
        #df = df.append(pd.Series([company_name]+info_tag_line+[company_loc],index=columns),ignore_index=True)
  i += 1      

df.to_csv("wanted.csv", sep=',', encoding = "UTF-8")

In [192]:
df

,회사이름,주요업무,자격요건,우대사항,혜택 및 복지,기술스택 ・ 툴,지역
0,[에비드넷],"• 의료 빅데이터 분석(데이터 전처리, 통계분석)• 의료 빅데이터 연구 기획","• 학사이상(통계, 수학, 산업공학, 컴퓨터 공학, 생명과학, 의료정보학 등)• 병...","• SAS, SPSS 등 분석툴 사용 가능• Python 사용 가능• 의학논문 작성...",[Work]• 시차출퇴근제도(8시~10시 출근)• 고사양 최신 업무용 PC 지원• ...,"[Python, R, SAS, SPSS]",경기도 성남시 분당구 판교로 323 벤처포럼빌딩 5층
1,[머큐리프로젝트],- AdTech 도메인에 특화된 데이터 분석 및 개발- 머신러닝/데이터 분석 기술을...,- 데이터 분석에 대한 팀이 존재 하지 않는 상황으로 분석을 위한 초기 인프라 구축...,- AdTech 도메인에 대한 지식을 보유하신 분- SQL을 능숙하게 다루실수 있는...,- 매월 통신비 비용 제공- 매년 1회 건강 검진 제공- 주3일 재택 근무 가능 (...,0,"서울시 강남구 봉은사로 38길 13, 3층, 4층, 5층"
2,[휴머스온],"데이터 가공, 분석 및 시각화- 온라인로그/캠페인 데이터 등을 활용하여 비즈니스 이...","- 데이터 분석 경험 1년 이상, 통계지식 보유- 마케팅전략/디지털마케팅전략/소비트...",- 통계 지식 보유- 데이터 엔지니어 및 데이터 사이언티스와 협업 능력 및 경험 보...,＜업무에만 집중 할 수 있는 근무환경을 만들고자 노력합니다＞ * 선릉역 3번출구 도...,"[Python, SQL, Google Analytics]","서울특별시 강남구 역삼2동 세방빌딩 10층, 12층"
3,[더블유클럽],• 서비스 지표 상승 및 문제 해결을 위한 데이터 기반의 인사이트 제공• 데이터 분...,• 단순한 아이디어 차원이 아닌 실질적인 Action 을 위한 구체적인 인사이트를 ...,"• ELK (Elastic Search, Kibana) 활용 경험이 있으신 분• G...",■W 기업문화- 대표를 포함하여 전직원 영어 닉네임 사용 (수평적인 사내 분위기)-...,"[Firebase, R, SQL, Excel, Google Analytics, El...",서울특별시 강남구 선릉로 428 (위워크타워)
4,[파이프트리스마트팜(PAIPTREESmartFarmInc.)],"- 농장 환경 및 생체 데이터 분석 및 주제 발굴- 정형 및 비정형 데이터(영상, ...","- 통계(빈도, 베이지안) 분석 및 모델링 경험- Shell, SQL 중급 이상, ...",- 통계 및 ML 관련 분야 석/박사 학위 또는 그에 준하는 경험- 로그 수준의 데...,- 8~10시 자율 출근- 4대보험 / 퇴직금 별도 지급- 점심 식대 지원 (1일 ...,"[Git, Linux, MySQL, C#, Node.js, VB .NET, ERP ...",서울특별시 강남구 봉은사로 125 리스트빌딩
...,...,...,...,...,...,...,...
68,[컬리],"• 분석을 위한 데이터를 설계 및 적재하고 대시보드, 알람 등의 프로덕트로 만들어 ...",• 유관 업무 경력 2년 이상을 선호합니다. 다만 기본 스킬이 충분하다면 경력이 조...,"• 커머스, 물류 또는 로그 분석 프로젝트의 경험이 있으신 분• 커머스, 물류 분야...","✔모두가 집중하고 성장할 수 있도록, 최적의 근무 환경을 제공해요✔• 리프레시도 필...","[Python, R, SQL, Data Analysis, Tableau]",서울특별시 강남구 테헤란로 133
69,[라인게임즈],"• 게임 로그 분석 및 통계 분석• 게임 데이터 분석을 통한, 문제점 파악 및 개선...",• 관련 경력 3년 이상 • 게임에 대한 열정과 및 게임 산업에 대한 이해가 있으...,• 게임 업계 혹은 타 업계 데이터 분석 실무 경험• R 혹은 Python을 이용한...,• 연 1회 프리미엄 건강검진 • 사내 전신 마사지실 및 헬스장• 사내 복지시설 내...,0,서울 강남구 테헤란로 218 AP 타워
70,[플라이셔(Flysher)],• 로그 설계 : 게임을 플레이하며 의사결정에 필요할 정보를 DB에 어떻게 쌓을지 ...,• 유관 업무 경력 3~5년인 분• SQL(Redshift)에 능숙한 분• 통계 지...,• 직무 관련 전공자 (컴퓨터공학/통계/수학 등)• Cloud(AWS 등) 개발 환...,[우리는 성과를 공유합니다]• 동종업계 상위 수준 연봉 보장• 각종 인센티브 제공•...,0,강남구 테헤란로 223 큰길타워 8층
71,[콘텐츠웨이브(wavve)],"• 데이터 분석/이해에 기반하여, 가입자 매출/증대를 위한 CRM기획 및 운영 ...",• 데이터 분석 유관 경력 7년 이상자• 데이터기반 가설수립 및 문제해결 역량• S...,"• 구독서비스 업계 경력(OTT, 뮤직 등)",“워라밸–Work & Life Balance” 일과 삶의 균형이 있는 회사 • 코로...,"[SAS, SQL, Excel, Google Analytics, Tableau, S...",서울 영등포구 여의나루로 60 포스트타워


In [113]:
#기술스텍불러오기
#info_tag.find_next_sibling().text//
list(map(lambda x:x.text,info_tag.next_element.next_element.findChild().findChildren()))

['Python', 'SQL', 'Google Analytics']

In [ ]:
info_tag_line = info_tag.next_sibling
info_tag_line.text

'- 매월 통신비 비용 제공- 매년 1회 건강 검진 제공- 주3일 재택 근무 가능 (협의시)'

In [112]:
# 회사 주소 불러오기
import json
loc = com_html.find('script',{'type':'application/ld+json'})
loc2 = dict(json.loads(loc.text))['jobLocation']['address']['streetAddress']
print(loc)

<script type="application/ld+json">{"@context":"https://schema.org","@type":"JobPosting","title":"데이터 분석가","description":"데이터 가공, 분석 및 시각화\n- 온라인로그/캠페인 데이터 등을 활용하여 비즈니스 이슈에 대해 가설을 수립\n- 데이터 핸들링 및 분석\n- 리포트 작성, 대시보드 기획/관리 등 시각화","url":"https://www.wanted.co.kr/wd/97358","hiringOrganization":{"@type":"Organization","name":"휴머스온","logo":"https://static.wanted.co.kr/images/wdes/0_4.5ae2c9e8.jpg"},"datePosted":"2022-07-04","industry":"IT, 컨텐츠","occupationalCategory":"Management & Business, Data Analyst,데이터 분석가","validThrough":"2022-08-01","employmentType":"FULL_TIME","jobLocation":{"@type":"Place","address":{"@type":"PostalAddress","addressRegion":"서울","addressCountry":"KR","streetAddress":"서울특별시 강남구 역삼2동 세방빌딩 10층, 12층"}},"experienceRequirements":["경력"]}</script>


In [ ]:
# 회사 이름 불러오기
print(h5.text)

머큐리프로젝트


In [ ]:
#'주요업무', '자격요건', '우대사항', '혜택 및 복지' 불러오기
apply = h6.next_sibling
print(apply.text)
#print(apply.text)
#a = apply.text.split('-')
#print(a)
    

- 데이터 분석에 대한 팀이 존재 하지 않는 상황으로 분석을 위한 초기 인프라 구축이 가능하신분- 3년 이상 실질적인 비즈니스 문제를 해결하기 위한 기법 또는 머신러닝을 적용한 경력을 보유한 분- 서비스를 개선하는 데 도움이 되는 논문이나 기술들을 빠르게 학습하여 프로토타이핑할 수 있는 문제해결 능력을 갖추신 분- 지원시 포트폴리오 필수 첨부(PPT, PDF, URL 등 자유형식)


In [ ]:
# from google.colab import drive # 구글 드라이브에 csv파일을 저장하기 위함

# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# f = open('/content/gdrive/My Drive/Colab Notebooks/txt_to_csv.txt', "w")
# f.write(apply.text)
# f.close()

In [ ]:
#pip install pandas

In [ ]:
# import pandas as pd

# file = pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/txt_to_csv.txt')
# new_csv_file = file.to_csv(r'/content/gdrive/My Drive/Colab Notebooks/txt_to_csv.csv', encoding='utf-8')

In [189]:
df.duplicated()

TypeError: ignored

In [170]:
df['지역'] = df['지역'].apply(lambda x: dict(json.loads(x.text))['jobLocation']['address']['streetAddress'])

In [197]:
def name(x):
  if x:
    return x.text

df['회사이름'] = df['회사이름'].apply(name)

In [182]:
keyword_list[df[df['회사이름'].isnull()].index[0]-1:df[df['회사이름'].isnull()].index[0]+2]

['/wd/65289', '/wd/90183', '/wd/63921']

In [198]:
df

,회사이름,주요업무,자격요건,우대사항,혜택 및 복지,기술스택 ・ 툴,지역
0,에비드넷,"• 의료 빅데이터 분석(데이터 전처리, 통계분석)• 의료 빅데이터 연구 기획","• 학사이상(통계, 수학, 산업공학, 컴퓨터 공학, 생명과학, 의료정보학 등)• 병...","• SAS, SPSS 등 분석툴 사용 가능• Python 사용 가능• 의학논문 작성...",[Work]• 시차출퇴근제도(8시~10시 출근)• 고사양 최신 업무용 PC 지원• ...,"[Python, R, SAS, SPSS]",경기도 성남시 분당구 판교로 323 벤처포럼빌딩 5층
1,머큐리프로젝트,- AdTech 도메인에 특화된 데이터 분석 및 개발- 머신러닝/데이터 분석 기술을...,- 데이터 분석에 대한 팀이 존재 하지 않는 상황으로 분석을 위한 초기 인프라 구축...,- AdTech 도메인에 대한 지식을 보유하신 분- SQL을 능숙하게 다루실수 있는...,- 매월 통신비 비용 제공- 매년 1회 건강 검진 제공- 주3일 재택 근무 가능 (...,0,"서울시 강남구 봉은사로 38길 13, 3층, 4층, 5층"
2,휴머스온,"데이터 가공, 분석 및 시각화- 온라인로그/캠페인 데이터 등을 활용하여 비즈니스 이...","- 데이터 분석 경험 1년 이상, 통계지식 보유- 마케팅전략/디지털마케팅전략/소비트...",- 통계 지식 보유- 데이터 엔지니어 및 데이터 사이언티스와 협업 능력 및 경험 보...,＜업무에만 집중 할 수 있는 근무환경을 만들고자 노력합니다＞ * 선릉역 3번출구 도...,"[Python, SQL, Google Analytics]","서울특별시 강남구 역삼2동 세방빌딩 10층, 12층"
3,더블유클럽,• 서비스 지표 상승 및 문제 해결을 위한 데이터 기반의 인사이트 제공• 데이터 분...,• 단순한 아이디어 차원이 아닌 실질적인 Action 을 위한 구체적인 인사이트를 ...,"• ELK (Elastic Search, Kibana) 활용 경험이 있으신 분• G...",■W 기업문화- 대표를 포함하여 전직원 영어 닉네임 사용 (수평적인 사내 분위기)-...,"[Firebase, R, SQL, Excel, Google Analytics, El...",서울특별시 강남구 선릉로 428 (위워크타워)
4,파이프트리스마트팜(PAIPTREESmartFarmInc.),"- 농장 환경 및 생체 데이터 분석 및 주제 발굴- 정형 및 비정형 데이터(영상, ...","- 통계(빈도, 베이지안) 분석 및 모델링 경험- Shell, SQL 중급 이상, ...",- 통계 및 ML 관련 분야 석/박사 학위 또는 그에 준하는 경험- 로그 수준의 데...,- 8~10시 자율 출근- 4대보험 / 퇴직금 별도 지급- 점심 식대 지원 (1일 ...,"[Git, Linux, MySQL, C#, Node.js, VB .NET, ERP ...",서울특별시 강남구 봉은사로 125 리스트빌딩
...,...,...,...,...,...,...,...
68,컬리,"• 분석을 위한 데이터를 설계 및 적재하고 대시보드, 알람 등의 프로덕트로 만들어 ...",• 유관 업무 경력 2년 이상을 선호합니다. 다만 기본 스킬이 충분하다면 경력이 조...,"• 커머스, 물류 또는 로그 분석 프로젝트의 경험이 있으신 분• 커머스, 물류 분야...","✔모두가 집중하고 성장할 수 있도록, 최적의 근무 환경을 제공해요✔• 리프레시도 필...","[Python, R, SQL, Data Analysis, Tableau]",서울특별시 강남구 테헤란로 133
69,라인게임즈,"• 게임 로그 분석 및 통계 분석• 게임 데이터 분석을 통한, 문제점 파악 및 개선...",• 관련 경력 3년 이상 • 게임에 대한 열정과 및 게임 산업에 대한 이해가 있으...,• 게임 업계 혹은 타 업계 데이터 분석 실무 경험• R 혹은 Python을 이용한...,• 연 1회 프리미엄 건강검진 • 사내 전신 마사지실 및 헬스장• 사내 복지시설 내...,0,서울 강남구 테헤란로 218 AP 타워
70,플라이셔(Flysher),• 로그 설계 : 게임을 플레이하며 의사결정에 필요할 정보를 DB에 어떻게 쌓을지 ...,• 유관 업무 경력 3~5년인 분• SQL(Redshift)에 능숙한 분• 통계 지...,• 직무 관련 전공자 (컴퓨터공학/통계/수학 등)• Cloud(AWS 등) 개발 환...,[우리는 성과를 공유합니다]• 동종업계 상위 수준 연봉 보장• 각종 인센티브 제공•...,0,강남구 테헤란로 223 큰길타워 8층
71,콘텐츠웨이브(wavve),"• 데이터 분석/이해에 기반하여, 가입자 매출/증대를 위한 CRM기획 및 운영 ...",• 데이터 분석 유관 경력 7년 이상자• 데이터기반 가설수립 및 문제해결 역량• S...,"• 구독서비스 업계 경력(OTT, 뮤직 등)",“워라밸–Work & Life Balance” 일과 삶의 균형이 있는 회사 • 코로...,"[SAS, SQL, Excel, Google Analytics, Tableau, S...",서울 영등포구 여의나루로 60 포스트타워


In [200]:
df.to_csv("wanted.csv", sep=',', encoding = "UTF-8")

In [201]:
df.to_excel("wanted_re.xls")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  """Entry point for launching an IPython kernel.


In [199]:
pd.read_csv("wanted.csv")

,Unnamed: 0,회사이름,주요업무,자격요건,우대사항,혜택 및 복지,기술스택 ・ 툴,지역
0,0,<h5>에비드넷</h5>,"• 의료 빅데이터 분석(데이터 전처리, 통계분석)• 의료 빅데이터 연구 기획","• 학사이상(통계, 수학, 산업공학, 컴퓨터 공학, 생명과학, 의료정보학 등)• 병...","• SAS, SPSS 등 분석툴 사용 가능• Python 사용 가능• 의학논문 작성...",[Work]• 시차출퇴근제도(8시~10시 출근)• 고사양 최신 업무용 PC 지원• ...,"['Python', 'R', 'SAS', 'SPSS']",경기도 성남시 분당구 판교로 323 벤처포럼빌딩 5층
1,1,<h5>머큐리프로젝트</h5>,- AdTech 도메인에 특화된 데이터 분석 및 개발- 머신러닝/데이터 분석 기술을...,- 데이터 분석에 대한 팀이 존재 하지 않는 상황으로 분석을 위한 초기 인프라 구축...,- AdTech 도메인에 대한 지식을 보유하신 분- SQL을 능숙하게 다루실수 있는...,- 매월 통신비 비용 제공- 매년 1회 건강 검진 제공- 주3일 재택 근무 가능 (...,0,"서울시 강남구 봉은사로 38길 13, 3층, 4층, 5층"
2,2,<h5>휴머스온</h5>,"데이터 가공, 분석 및 시각화- 온라인로그/캠페인 데이터 등을 활용하여 비즈니스 이...","- 데이터 분석 경험 1년 이상, 통계지식 보유- 마케팅전략/디지털마케팅전략/소비트...",- 통계 지식 보유- 데이터 엔지니어 및 데이터 사이언티스와 협업 능력 및 경험 보...,＜업무에만 집중 할 수 있는 근무환경을 만들고자 노력합니다＞ * 선릉역 3번출구 도...,"['Python', 'SQL', 'Google Analytics']","서울특별시 강남구 역삼2동 세방빌딩 10층, 12층"
3,3,<h5>더블유클럽</h5>,• 서비스 지표 상승 및 문제 해결을 위한 데이터 기반의 인사이트 제공• 데이터 분...,• 단순한 아이디어 차원이 아닌 실질적인 Action 을 위한 구체적인 인사이트를 ...,"• ELK (Elastic Search, Kibana) 활용 경험이 있으신 분• G...",■W 기업문화- 대표를 포함하여 전직원 영어 닉네임 사용 (수평적인 사내 분위기)-...,"['Firebase', 'R', 'SQL', 'Excel', 'Google Anal...",서울특별시 강남구 선릉로 428 (위워크타워)
4,4,<h5>파이프트리스마트팜(PAIPTREESmartFarmInc.)</h5>,"- 농장 환경 및 생체 데이터 분석 및 주제 발굴- 정형 및 비정형 데이터(영상, ...","- 통계(빈도, 베이지안) 분석 및 모델링 경험- Shell, SQL 중급 이상, ...",- 통계 및 ML 관련 분야 석/박사 학위 또는 그에 준하는 경험- 로그 수준의 데...,- 8~10시 자율 출근- 4대보험 / 퇴직금 별도 지급- 점심 식대 지원 (1일 ...,"['Git', 'Linux', 'MySQL', 'C#', 'Node.js', 'VB...",서울특별시 강남구 봉은사로 125 리스트빌딩
...,...,...,...,...,...,...,...,...
68,68,<h5>컬리</h5>,"• 분석을 위한 데이터를 설계 및 적재하고 대시보드, 알람 등의 프로덕트로 만들어 ...",• 유관 업무 경력 2년 이상을 선호합니다. 다만 기본 스킬이 충분하다면 경력이 조...,"• 커머스, 물류 또는 로그 분석 프로젝트의 경험이 있으신 분• 커머스, 물류 분야...","✔모두가 집중하고 성장할 수 있도록, 최적의 근무 환경을 제공해요✔• 리프레시도 필...","['Python', 'R', 'SQL', 'Data Analysis', 'Table...",서울특별시 강남구 테헤란로 133
69,69,<h5>라인게임즈</h5>,"• 게임 로그 분석 및 통계 분석• 게임 데이터 분석을 통한, 문제점 파악 및 개선...",• 관련 경력 3년 이상 • 게임에 대한 열정과 및 게임 산업에 대한 이해가 있으...,• 게임 업계 혹은 타 업계 데이터 분석 실무 경험• R 혹은 Python을 이용한...,• 연 1회 프리미엄 건강검진 • 사내 전신 마사지실 및 헬스장• 사내 복지시설 내...,0,서울 강남구 테헤란로 218 AP 타워
70,70,<h5>플라이셔(Flysher)</h5>,• 로그 설계 : 게임을 플레이하며 의사결정에 필요할 정보를 DB에 어떻게 쌓을지 ...,• 유관 업무 경력 3~5년인 분• SQL(Redshift)에 능숙한 분• 통계 지...,• 직무 관련 전공자 (컴퓨터공학/통계/수학 등)• Cloud(AWS 등) 개발 환...,[우리는 성과를 공유합니다]• 동종업계 상위 수준 연봉 보장• 각종 인센티브 제공•...,0,강남구 테헤란로 223 큰길타워 8층
71,71,<h5>콘텐츠웨이브(wavve)</h5>,"• 데이터 분석/이해에 기반하여, 가입자 매출/증대를 위한 CRM기획 및 운영 ...",• 데이터 분석 유관 경력 7년 이상자• 데이터기반 가설수립 및 문제해결 역량• S...,"• 구독서비스 업계 경력(OTT, 뮤직 등)",“워라밸–Work & Life Balance” 일과 삶의 균형이 있는 회사 • 코로...,"['SAS', 'SQL', 'Excel', 'Google Analytics', 'T...",서울 영등포구 여의나루로 60 포스트타워
